## Stage 1: Installing dependencies and setting up the environment

In [ ]:
#!pip install tensorflow-gpu==2.0.0.alpha0

## Stage 2: Importing project dependencies

In [1]:
import time
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.7.0'

## Stage 3: Dataset preprocessing

### Loading the MNIST dataset

In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


### Image normalization

In [4]:
X_train = X_train / 255.
X_test = X_test / 255.

In [5]:
X_train.shape

(60000, 28, 28)

### Dataset reshaping

In [6]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [7]:
X_train.shape

(60000, 784)

## Stage 4: Distributed Training

### Defining normal (non distributed) model

In [8]:
model_normal = tf.keras.models.Sequential()

In [9]:
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))

In [10]:
model_normal.add(tf.keras.layers.Dropout(rate=0.2))

In [11]:
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [12]:
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Defining the Distributed Strategy

In [13]:
distribute = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


### Defining a distributed model

In [14]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


### Speed comparison between normal training and distributed training process

In [15]:
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=10, batch_size=25)
print("Distributed training took: {}".format(time.time() - start_time))

Epoch 1/10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
2400/2400 [==============================] - 14s 4ms/step - loss: 0.2839 - sparse_categorical_accuracy: 0.9169
Epoch 2/10
2400/2400 [================

In [16]:
start_time = time.time()
model_normal.fit(X_train, y_train, epochs=10, batch_size=25)
print("Normal training took: {}".format(time.time() - start_time))

Epoch 1/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.2836 - sparse_categorical_accuracy: 0.9176
Epoch 2/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.1395 - sparse_categorical_accuracy: 0.9586
Epoch 3/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.1058 - sparse_categorical_accuracy: 0.9684
Epoch 4/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.0874 - sparse_categorical_accuracy: 0.9726
Epoch 5/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.0733 - sparse_categorical_accuracy: 0.9768
Epoch 6/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.0656 - sparse_categorical_accuracy: 0.9785
Epoch 7/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.0582 - sparse_categorical_accuracy: 0.9811
Epoch 8/10
2400/2400 [==============================] - 7s 3ms/step - loss: 0.0518 - sparse_categorical_accuracy: 0.9826
Epoch 9/10
2400/2400 [==========